In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import src.urban_diff_helper as ud_helper
import pandas as pd

# load geometries

In [48]:
zms, zms_agebs, zms_tvor, zms_grids, mg_mapping = ud_helper.load_geoms()

loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson
reading existing grids
reading existing grids
dividing grids by municipality
computing the intersection between p1 grid and p2 mgmu
computing area of the intersections
computing the weight
reading existing grids
reading existing grids
dividing grids by municipality
computing the intersection between p1 grid and p2 mgmu
computing area of the intersections
computing the weight


# load interpolation and interpolate avg

In [16]:
avg_tw, avg_a, avg_g, avg_idw = ud_helper.interpolation()

loading cached tw average stats/stat_tw_dow_aver_hr_uniq_user-out+in.pickle
loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson
computing the intersection between p1 tower and p2 ageb
computing area of the intersections
computing the weight
loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson
computing the intersection between p1 tower and p2 ageb
computing area of the intersections
computing the weight
reading existing grids
loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson
computing the intersection between p1 tower and p2 grid
computing area of the intersections
computing the weight
reading existing grids
dividing grids by municipality
computing the intersection between p1 grid and p2 mgmu
computing area of the intersections
computing the weight
loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson
computing the intersection between p1 tower and p2 grid
computing area of the inters

# draw function

In [7]:
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
import plotly.graph_objects as go

In [143]:
def draw_scatter(data_df, mode='lines+markers',height=600,margin_r=100):
    
    data = [
        go.Scatter(x='zm' + data_df.index.astype(str),
                   y=data_df[intpl_type].tolist(),
                   name=str(intpl_type),
                   mode='lines+markers'
                  ) for intpl_type in data_df
    ]
    fig = go.Figure()
    for trace in data:
        fig.add_trace(trace)
    fig.update_layout(height=height,margin=dict(l=10, r=margin_r, t=10, b=10),)

    offline.iplot(fig)

# urban dilatation index

In [20]:
print('computing dv_a')
dv_a = {}
for by, avg in avg_a.items():
    dv_a[by] = ud_helper.dilatation.dv_for_mpa_ageb(avg, zms, zms_agebs)

print('computing dv_g')
dv_g = {}
for (side, by, per_mun), avg in avg_g.items():
    zms_g = zms_grids[(side, per_mun)]
    dv_g[(side, by, per_mun)] = ud_helper.dilatation.dv_for_mpa_grids(avg, zms, zms_g)

print('computing dv_idw')
dv_idw = {}
for key, avg in avg_idw.items():
    zms_g = zms_grids[key]
    dv_idw[key] = ud_helper.dilatation.dv_for_mpa_grids(avg, zms, zms_g)

computing dv_a
computing dv_g
computing dv_idw


In [38]:
dv_idw.keys()

dict_keys([(1000, False), (1000, True), (2000, False), (2000, True)])

In [23]:
dc = 'dilatation coefficient'


In [43]:
a= {f'g{side//1000}_{by}_{per_mun}':dv[dc] for (side, by, per_mun), dv in dv_g.items() if not per_mun}
b= {f'idw10_g{side//1000}_{per_mun}':dv[dc] for (side, per_mun), dv in dv_idw.items() if not per_mun}


In [44]:
dc = 'dilatation coefficient'
dila_coef = pd.DataFrame({
    'ageb_area': dv_a['area'][dc],
    'ageb_pop': dv_a['pop'][dc],
    ** a, **b
})
sort_col = ud_helper.zms_sort_cols[0]
dila_coef = dila_coef.loc[zms.sort_values(sort_col).index]

# sort_col = dila_coef.columns[0]
# dila_coef=dila_coef.sort_values(sort_col)
# print(sort_col)

draw_scatter(dila_coef, height=300)
draw_scatter(dila_coef.std(axis=1).to_frame(), height=300, margin_r=135)

# hotspots

In [49]:
hotspot_type = 'loubar'
n_hs_a = {}
comp_coef_a = {}
for by in ['area', 'pop']:
    for per_mun in [False, True]:
        print(key,end=' ')
        key = (by, per_mun)
        n, cc = ud_helper.ftr_hs.hs_stats_ageb(
            avg_a[by], zms, zms_agebs, mg_mapping, per_mun, hotspot_type)
        n_hs_a[key] = n
        comp_coef_a[key] = cc

working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 


In [112]:
n_hs_g = {}
comp_coef_g = {}
for key, avg in avg_g.items():
    print(key,end=' ')
    side, by, per_mun = key
    zms_g = zms_grids[(side, per_mun)]
    n, cc = ud_helper.ftr_hs.hs_stats_grid(
        avg, zms, zms_g, per_mun, hotspot_type)
    n_hs_g[key] = n
    comp_coef_g[key] = cc

(1000, 'area', False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(1000, 'area', True) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(1000, 'pop', False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(1000, 'pop', True) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(2000, 'area', False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 5

In [116]:
n_hs_idw = {}
comp_coef_idw = {}
for key, avg in avg_idw.items():
    print(key,end=' ')
    side, per_mun = key
    zms_g = zms_grids[key]
    n, cc = ud_helper.ftr_hs.hs_stats_grid(
        avg, zms, zms_g, per_mun, hotspot_type)
    n_hs_idw[key] = n
    comp_coef_idw[key] = cc

(1000, False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(1000, True) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(2000, False) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
(2000, True) working on 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 


In [158]:
per_mun_str = lambda x: 'zm' if not x else 'mun'

a = {('ageb', None, by, per_mun_str(per_mun)):cc for (by, per_mun), cc in comp_coef_a.items() }
g = {('g',str(side//1000),by,per_mun_str(per_mun)): cc for (side,by,per_mun), cc in comp_coef_g.items()}
idw = {('idw10', str(side//1000), None, per_mun_str(per_mun)): cc for (side,per_mun), cc in comp_coef_idw.items()}

In [159]:
comp_coef = pd.DataFrame({
**a, **g, ** idw
})

sort_col = ud_helper.zms_sort_cols[3]
comp_coef = comp_coef.loc[zms.sort_values(sort_col).index]
sort_col = comp_coef.columns[0]
comp_coef=comp_coef.sort_values(sort_col)
print(sort_col)

draw_scatter(comp_coef, height=300)
draw_scatter(comp_coef.std(axis=1).to_frame(), height=300, margin_r=175)

('ageb', nan, 'area', 'zm')


In [162]:
per_mun_str = lambda x: 'zm' if not x else 'mun'

a = {('ageb', None, by, per_mun_str(per_mun)):cc for (by, per_mun), cc in n_hs_a.items() }
g = {('g',str(side//1000),by,per_mun_str(per_mun)): cc for (side,by,per_mun), cc in n_hs_g.items()}
idw = {('idw10', str(side//1000), None, per_mun_str(per_mun)): cc for (side,per_mun), cc in n_hs_idw.items()}

Mean_number_hot = pd.DataFrame({
**a, **g, **idw
})
sort_col = Mean_number_hot.columns[0]
Mean_number_hot=Mean_number_hot.sort_values(sort_col)
print(sort_col)

draw_scatter(Mean_number_hot)

('ageb', nan, 'area', 'zm')
